*weekly(version 11/11) 1.23901  >> price volume instead of counting
* weekly(version 9/9) 1.08116
* weekly(version 7/9) 1.08567
* weekly(version 6/9) 1.27601
* weekly(version 4/9) 1.24642
* weekly(version 3/9) 1.08348
* weekly(version 2/9) 1.06151
* weekly(version 1/9) 1.04857


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
print(os.listdir("../input"))

items_df = pd.read_csv('../input/items.csv')
shops_df = pd.read_csv('../input/shops.csv')
icats_df = pd.read_csv('../input/item_categories.csv')
train_df = pd.read_csv('../input/sales_train.csv')
smpsb_df = pd.read_csv('../input/sample_submission.csv')
test_df  = pd.read_csv('../input/test.csv')

# Any results you write to the current directory are saved as output.
train_df['date'] = pd.to_datetime(train_df.date,format="%d.%m.%Y")
train_df['week'] = ( train_df.date.values.astype('datetime64[W]'))
train_df['volum'] = train_df['item_cnt_day']*train_df['item_price']
train_df.head()

# week sum of sales per shop and per item
its not used here actually

In [2]:
## Pivot by monht to wide format
p_df = train_df.pivot_table(index=['shop_id','item_id'], columns='week', values='item_cnt_day',aggfunc='sum').fillna(0.0)
pv_df = train_df.pivot_table(index=['shop_id','item_id'], columns='week', values='volum',aggfunc='sum').fillna(0.0)
p_df.head()

# deseasonalize sale per item per month
append sales per item_month to p_df

In [3]:
#diff = p_df.T.diff().dropna()
#exp_12=p_df.T.ewm(span=12,adjust=False).mean()

##   i_df = train_df.pivot_table(index=['item_id'], columns='date_block_num', values='item_cnt_day',aggfunc='sum').fillna(0.0)
#i_df = train_df.pivot_table(index=['item_id'], columns='date_block_num', values='volum',aggfunc='sum').fillna(0.0)
i_df = train_df.pivot_table(index=['item_id'], columns='week', values='item_cnt_day',aggfunc='sum').fillna(0.0)

is_df=(i_df.T/i_df.T.max()).T
is_df['imax']=i_df.T.max()
pis_df=pd.DataFrame()
pis_df['isom']=pv_df.sum(axis=1)
pis_df=pis_df.reset_index()
pis_df=pis_df.merge(is_df,how='left',left_on=[ "item_id"],right_on=["item_id"])

pis_df.head()

# deseasonalize sales per shop per month
append sales per shop_month to p_df

what we could do further is add a salesforecast for period 34 with ARIMA

In [4]:
s_df = train_df.pivot_table(index=['shop_id'], columns='week', values='item_cnt_day',aggfunc='sum').fillna(0.0)
##       s_df = train_df.pivot_table(index=['shop_id'], columns='date_block_num', values='item_cnt_day',aggfunc='sum').fillna(0.0)
#sv_df = train_df.pivot_table(index=['shop_id'], columns='date_block_num', values='volum',aggfunc='sum').fillna(0.0)
ss_df=(s_df.T/s_df.T.max()).T
#ssv_df=(sv_df.T/sv_df.T.max()).T
ss_df['smax']=s_df.T.max()
#ss_df['svmax']=sv_df.T.max()

pss_df=pd.DataFrame()
pss_df['ssom']=pv_df.sum(axis=1)
pss_df=pss_df.reset_index()
pss_df=pss_df.merge(ss_df,how='left',left_on=[ "shop_id"],right_on=["shop_id"])
#pss_df=pss_df.merge(ssv_df,how='left',left_on=[ "shop_id"],right_on=["shop_id"])
pss_df.head()

# cleanup
make abit orderly

In [5]:
train_cleaned_df=pis_df.merge(pss_df,how='inner',on=['shop_id','item_id'])
train_cleaned_df.head()

In [6]:
kolom=train_cleaned_df.columns
kolom=[  x  for x in kolom if x not in ['shop_id', 'item_id', 'isom','ssom','imax','smax','svmax'] ]
train_cleaned_df=train_cleaned_df[['shop_id', 'item_id', 'isom','ssom','imax','smax'] + list(kolom)]
#train_cleaned_df=train_cleaned_df[['shop_id', 'item_id', 'isom','ssom','imax','smax','svmax'] + list(kolom)]
train_cleaned_df['shop_id']= train_cleaned_df.shop_id.astype('str')
train_cleaned_df['item_id']= train_cleaned_df.item_id.astype('str')
train_cleaned_df.shape

# training 
checking forecast in scatter plot, looks excellent

In [7]:
import xgboost as xgb
param = {'max_depth':10, 
         'subsample':1,
         'min_child_weight':0.5,
         'eta':0.3, 
         'num_round':1000, 
         'seed':1,
         'silent':0,
         'eval_metric':'rmse'}

progress = dict()
xgbtrain = xgb.DMatrix(train_cleaned_df.iloc[:, :].values, p_df.iloc[:, -5:].sum(axis=1).values)
watchlist  = [(xgbtrain,'train-rmse')]

bst = xgb.train(param, xgbtrain)
preds = bst.predict(xgb.DMatrix(train_cleaned_df.iloc[:,:].values))
from sklearn.metrics import mean_squared_error 
rmse = np.sqrt(mean_squared_error(preds,p_df.iloc[:, -5:].sum(axis=1).values))
print(rmse)

plt.scatter(preds, p_df.iloc[:, -5:].sum(axis=1).values, marker='.',s=1)


# try implement the knowledge to the testdata
so the funny thing is we dont' forecast yet, we simply forecast october on the test data

In [8]:
apply_df = test_df
apply_df['shop_id']= apply_df.shop_id.astype('str')
apply_df['item_id']= apply_df.item_id.astype('str')
apply_df = test_df.merge(train_cleaned_df, how = "left", on = ["shop_id", "item_id"]).fillna(0.0)
apply_df.drop('ID',axis=1).head(),apply_df.drop('ID',axis=1).shape

In [9]:
# Move to one month front
#d = dict(zip(apply_df.columns[4:],list([-1,-1,])+list(np.array(list(apply_df.columns[6:])) )))

#apply_df  = apply_df.rename(d, axis = 1)
#preds = bst.predict(xgb.DMatrix(apply_df.iloc[:, (apply_df.columns != 'ID') & (apply_df.columns != -1)].values))

preds = bst.predict(xgb.DMatrix(apply_df.drop('ID',axis=1).values))

In [10]:
preds

In [ ]:
preds=preds

# Normalize prediction to [0-20]
preds = list(map(lambda x: min(20,max(x,0)), list(preds)))
sub_df = pd.DataFrame({'ID':apply_df.ID,'item_cnt_month': preds })
pd.DataFrame.hist(sub_df[sub_df['item_cnt_month']>1.7])

In [35]:
sub_df.to_csv('xg_boost4_cats.csv',index=False)